In [1]:
from huggingface_hub import snapshot_download

# Define the model repo
model_name = "Deva8/Skins"

# Download the model locally
snapshot_download(repo_id=model_name, local_dir="/content/")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

HAM10000_images_part_main.zip:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

final.zip:   0%|          | 0.00/103M [00:00<?, ?B/s]

segmented_images.zip:   0%|          | 0.00/125M [00:00<?, ?B/s]

HAM10000_metadata.csv:   0%|          | 0.00/563k [00:00<?, ?B/s]

'/content'

In [2]:
from huggingface_hub import hf_hub_download

hf_hub_download(
    repo_id="Deva8/Skins",
    filename="HAM10000_metadata.csv",  # Replace with the exact file name
    local_dir="/content/",    # Optional: specify the local directory
)


'/content/HAM10000_metadata.csv'

In [3]:
import shutil

shutil.unpack_archive("/content/HAM10000_images_part_main.zip", "/content/")


In [4]:
import shutil

shutil.unpack_archive("/content/segmented_images.zip", "/content/")


In [5]:
import torch
print(torch.version.cuda)          # Should show 12.6
print(torch.cuda.is_available())   # Should return True

12.4
False


In [6]:
!kaggle datasets download -d tschandl/ham10000-lesion-segmentations

Dataset URL: https://www.kaggle.com/datasets/tschandl/ham10000-lesion-segmentations
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
 97% 10.0M/10.3M [00:01<00:00, 12.5MB/s]
100% 10.3M/10.3M [00:01<00:00, 7.32MB/s]


In [ ]:
!unzip /content/ham10000-lesion-segmentations.zip -d ham10000_segmentations

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class SkinLesionDataset(Dataset):
    """Dataset class for skin lesion classification with advanced augmentation pipeline."""
    def __init__(self, image_dir, mask_dir, metadata_df, transform=None, phase='train'):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.metadata_df = metadata_df
        self.phase = phase

        # Define class mapping for different skin lesion types
        self.class_map = {
            'akiec': 0,  # Actinic Keratosis
            'bcc': 1,    # Basal Cell Carcinoma
            'bkl': 2,    # Benign Keratosis
            'df': 3,     # Dermatofibroma
            'mel': 4,    # Melanoma
            'nv': 5,     # Melanocytic Nevus
            'vasc': 6    # Vascular Lesion
        }

        # Set up transformations
        self.transform = transform if transform is not None else self.get_transforms()

    def get_transforms(self):
        """Define the augmentation pipeline based on the dataset phase."""
        if self.phase == 'train':
            return A.Compose([
                # Spatial augmentations
                A.RandomRotate90(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.Affine(
                    scale=(0.9, 1.1),
                    rotate=(-45, 45),
                    translate_percent=(-0.0625, 0.0625),
                    p=0.5
                ),

                # Color augmentations
                A.OneOf([
                    A.RandomBrightnessContrast(
                        brightness_limit=0.2,
                        contrast_limit=0.2,
                        p=0.5
                    ),
                    A.HueSaturationValue(
                        hue_shift_limit=20,
                        sat_shift_limit=30,
                        val_shift_limit=20,
                        p=0.5
                    ),
                ], p=0.5),

                # Normalization
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                ),
                ToTensorV2()
            ])
        else:
            return A.Compose([
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                ),
                ToTensorV2()
            ])

    def find_mask_file(self, image_id):
        """Try different mask naming conventions to find the correct mask file."""
        mask_patterns = [
            f"{image_id}_segmentation.png",
            f"{image_id}_segmented.png",
        ]

        for pattern in mask_patterns:
            mask_path = os.path.join(self.mask_dir, pattern)
            if os.path.exists(mask_path):
                return mask_path

        return None

    def apply_mask(self, image, mask):
        """Apply segmentation mask to isolate the lesion area."""
        binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
        binary_mask = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
        masked_image = cv2.bitwise_and(image, binary_mask)
        return masked_image

    def __getitem__(self, idx):
        """Get a single item from the dataset with augmentations applied."""
        row = self.metadata_df.iloc[idx]
        image_id = row['image_id']
        label = self.class_map[row['dx']]

        try:
            # Load image
            image_path = os.path.join(self.image_dir, f"{image_id}.jpg")
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image file not found: {image_path}")

            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Find and load mask
            mask_path = self.find_mask_file(image_id)
            if mask_path is None:
                mask = np.full((224, 224), 255, dtype=np.uint8)
                logger.warning(f"No mask found for {image_id}, using full image")
            else:
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                if mask is None:
                    mask = np.full((224, 224), 255, dtype=np.uint8)
                    logger.warning(f"Failed to load mask for {image_id}, using full image")

            # Preprocess images
            image = cv2.resize(image, (224, 224))
            mask = cv2.resize(mask, (224, 224))
            masked_image = self.apply_mask(image, mask)

            # Apply augmentations
            augmented = self.transform(image=masked_image)
            augmented_image = augmented['image']

            return augmented_image, label, image_id

        except Exception as e:
            logger.error(f"Error processing {image_id}: {str(e)}")
            return torch.zeros((3, 224, 224)), label, image_id

    def __len__(self):
        return len(self.metadata_df)

class VGG50(nn.Module):
    """Custom implementation of VGG50 architecture."""
    def __init__(self, num_classes=7, dropout_rate=0.5):
        super(VGG50, self).__init__()

        # First block (4 conv layers)
        self.block1 = nn.Sequential(
            self._conv_block(3, 64),
            self._conv_block(64, 64),
            self._conv_block(64, 64),
            self._conv_block(64, 64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Second block (8 conv layers)
        self.block2 = nn.Sequential(
            self._conv_block(64, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            self._conv_block(128, 128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Third block (12 conv layers)
        self.block3 = nn.Sequential(
            self._conv_block(128, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            self._conv_block(256, 256),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fourth block (12 conv layers)
        self.block4 = nn.Sequential(
            self._conv_block(256, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fifth block (11 conv layers)
        self.block5 = nn.Sequential(
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            self._conv_block(512, 512),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Classifier (3 FC layers)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(4096, num_classes)
        )

        # Initialize weights
        self._initialize_weights()

    def _conv_block(self, in_channels, out_channels):
        """Creates a VGG-style convolutional block with batch normalization."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def _initialize_weights(self):
        """Initialize model weights using the strategy from the original VGG paper."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """Forward pass of the network."""
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class VGG50Trainer:
    """Utility class for training and evaluating VGG50 model."""
    def __init__(self, model, criterion, optimizer, device):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device

        # Learning rate scheduler for better convergence
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.1,
            patience=5,
            verbose=True
        )

    def train_epoch(self, train_loader):
        """Train the model for one epoch."""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, targets, _) in enumerate(train_loader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            # Gradient computation and optimization step
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            loss.backward()

            # Gradient clipping to prevent explosion
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

            self.optimizer.step()

            # Statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        accuracy = 100. * correct / total
        average_loss = running_loss / len(train_loader)
        return average_loss, accuracy

    def evaluate(self, val_loader):
        """Evaluate the model on validation set."""
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets, _ in val_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)

                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        accuracy = 100. * correct / total
        average_loss = running_loss / len(val_loader)
        return average_loss, accuracy

class MetricTracker:
    """Tracks and computes various training metrics during model training."""
    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all tracking variables to initial state."""
        self.running_loss = 0.0
        self.correct = 0
        self.total = 0
        self.batch_count = 0

    def update(self, loss, predictions, targets):
        """Update metrics with new batch results."""
        self.running_loss += loss.item()
        self.batch_count += 1
        self.total += targets.size(0)
        self.correct += predictions.eq(targets).sum().item()

    @property
    def loss(self):
        """Calculate average loss."""
        return self.running_loss / self.batch_count

    @property
    def accuracy(self):
        """Calculate accuracy percentage."""
        return 100. * self.correct / self.total

def train_model(train_loader, val_loader, model, num_epochs=50, device='cuda'):
    """
    Train the VGG50 model with advanced techniques including learning rate scheduling,
    gradient clipping, and early stopping.

    Args:
        train_loader (DataLoader): DataLoader for training data
        val_loader (DataLoader): DataLoader for validation data
        model (nn.Module): VGG50 model instance
        num_epochs (int): Maximum number of training epochs
        device (str): Device to train on ('cuda' or 'cpu')

    Returns:
        tuple: Trained model and training history dictionary
    """
    # Create directory for saving models with timestamp
    save_dir = os.path.join('models', datetime.now().strftime('%Y%m%d_%H%M%S'))
    os.makedirs(save_dir, exist_ok=True)

    # Initialize training components
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=3e-4,
        weight_decay=1e-4,
        betas=(0.9, 0.999)
    )

    # Learning rate scheduler with warmup
    scheduler = CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,
        eta_min=1e-6
    )

    # Early stopping parameters
    best_val_loss = float('inf')
    patience = 7  # Increased patience for more stable training
    patience_counter = 0

    # Training history dictionary
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'learning_rates': []
    }

    # Print training setup
    print("\nTraining Configuration:")
    print("=" * 80)
    print(f"Model: VGG50")
    print(f"Optimizer: AdamW (lr={optimizer.param_groups[0]['lr']})")
    print(f"Scheduler: CosineAnnealingLR")
    print(f"Max Epochs: {num_epochs}")
    print(f"Device: {device}")
    print(f"Early Stopping Patience: {patience}")

    # Training progress header
    print("\nTraining Progress:")
    print("=" * 80)
    print(f"{'Epoch':^10}{'Train Loss':^15}{'Train Acc':^15}{'Val Loss':^15}{'Val Acc':^15}{'LR':^10}")
    print("-" * 80)

    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_metrics = MetricTracker()

        # Progress bar for training batches
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)

        for images, labels, _ in train_pbar:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass with gradient clipping
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            # Update metrics
            _, predicted = outputs.max(1)
            train_metrics.update(loss, predicted, labels)

            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{train_metrics.loss:.4f}',
                'acc': f'{train_metrics.accuracy:.2f}%'
            })

        # Validation phase
        model.eval()
        val_metrics = MetricTracker()

        with torch.no_grad():
            for images, labels, _ in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                _, predicted = outputs.max(1)
                val_metrics.update(loss, predicted, labels)

        # Update learning rate
        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step()

        # Update and display metrics
        train_loss = train_metrics.loss
        train_acc = train_metrics.accuracy
        val_loss = val_metrics.loss
        val_acc = val_metrics.accuracy

        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['learning_rates'].append(current_lr)

        # Print progress
        print(f"{epoch+1:^10d}{train_loss:^15.4f}{train_acc:^15.2f}{val_loss:^15.4f}{val_acc:^15.2f}{current_lr:^10.2e}")

        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'val_loss': val_metrics.loss,
            'val_acc': val_metrics.accuracy,
            'history': history
        }

        # Save best model
        if val_metrics.loss < best_val_loss:
            best_val_loss = val_metrics.loss
            torch.save(checkpoint, os.path.join(save_dir, 'best_model.pth'))
            print(f"\nNew best model saved! (val_loss: {val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("\nEarly stopping triggered!")
                break

        # Save regular checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            torch.save(
                checkpoint,
                os.path.join(save_dir, f'checkpoint_epoch_{epoch + 1}.pth')
            )

    # Print final summary
    print("\nTraining Complete!")
    print("=" * 80)
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"Final training accuracy: {history['train_acc'][-1]:.2f}%")
    print(f"Final validation accuracy: {history['val_acc'][-1]:.2f}%")
    print(f"Model checkpoints saved in: {save_dir}")

    # Load best model state
    best_checkpoint = torch.load(os.path.join(save_dir, 'best_model.pth'))
    model.load_state_dict(best_checkpoint['state_dict'])

    return model, history

def evaluate_model(model, test_loader, device='cuda'):
    """
    Evaluate the model on a test set and compute detailed metrics with enhanced logging.

    Args:
        model (nn.Module): Trained model
        test_loader (DataLoader): Test data loader
        device (str): Device to evaluate on

    Returns:
        dict: Dictionary containing various evaluation metrics
    """
    model.eval()
    criterion = nn.CrossEntropyLoss()

    # Initialize tracking variables
    all_predictions = []
    all_labels = []
    test_loss = 0
    correct = 0
    total = 0

    # Initialize per-class tracking
    class_correct = [0] * 7
    class_total = [0] * 7

    # Class name mapping for better readability
    class_names = {
        0: 'Actinic Keratosis',
        1: 'Basal Cell Carcinoma',
        2: 'Benign Keratosis',
        3: 'Dermatofibroma',
        4: 'Melanoma',
        5: 'Melanocytic Nevus',
        6: 'Vascular Lesion'
    }

    print("\nStarting model evaluation...")

    with torch.no_grad():
        for images, labels, _ in tqdm(test_loader, desc='Evaluating'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Compute per-class accuracy
            for idx in range(len(labels)):
                label = labels[idx]
                pred = predicted[idx]
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate overall metrics
    test_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total

    # Calculate and print per-class accuracy
    class_accuracy = {}

    print("\nEvaluation Results:")
    print("=" * 80)
    print(f"Overall Test Accuracy: {accuracy:.2f}%")
    print(f"Overall Test Loss: {test_loss:.4f}")
    print("\nPer-class Accuracy:")
    print("-" * 80)
    print(f"{'Class ID':<10}{'Class Name':<30}{'Accuracy':<15}{'Samples':<10}")
    print("-" * 80)

    for i in range(7):
        if class_total[i] > 0:
            class_acc = 100. * class_correct[i] / class_total[i]
            class_accuracy[i] = class_acc
            print(f"{i:<10}{class_names[i]:<30}{class_acc:.2f}%{class_total[i]:<10}")
        else:
            class_accuracy[i] = 0
            print(f"{i:<10}{class_names[i]:<30}N/A{0:<10}")

    # Log results using the logger
    logger.info("Evaluation completed successfully")
    logger.info(f"Final test accuracy: {accuracy:.2f}%")
    logger.info(f"Final test loss: {test_loss:.4f}")
    logger.info("Per-class accuracy:")
    for class_id, acc in class_accuracy.items():
        logger.info(f"Class {class_id} ({class_names[class_id]}): {acc:.2f}%")

    return {
        'test_loss': test_loss,
        'accuracy': accuracy,
        'class_accuracy': class_accuracy,
        'predictions': np.array(all_predictions),
        'true_labels': np.array(all_labels)
    }

def predict_single_image(model, image_path, device='cuda'):
    """
    Make a prediction on a single image.

    Args:
        model (nn.Module): Trained model
        image_path (str): Path to the image file
        device (str): Device to run prediction on

    Returns:
        tuple: Predicted class and confidence score
    """
    # Class mapping for interpretation
    class_names = {
        0: 'Actinic Keratosis',
        1: 'Basal Cell Carcinoma',
        2: 'Benign Keratosis',
        3: 'Dermatofibroma',
        4: 'Melanoma',
        5: 'Melanocytic Nevus',
        6: 'Vascular Lesion'
    }

    try:
        # Load and preprocess image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))

        # Apply transforms
        transform = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

        augmented = transform(image=image)
        image_tensor = augmented['image'].unsqueeze(0).to(device)

        # Make prediction
        model.eval()
        with torch.no_grad():
            outputs = model(image_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidence, predicted = probabilities.max(1)

        predicted_class = predicted.item()
        confidence_score = confidence.item()

        return {
            'class_name': class_names[predicted_class],
            'class_id': predicted_class,
            'confidence': confidence_score
        }

    except Exception as e:
        logger.error(f"Error predicting image {image_path}: {str(e)}")
        return None

def main():
    """
    Main function to set up and run the training pipeline.
    Includes data preparation, model training, and evaluation.
    """
    try:
        # Set random seeds for reproducibility
        torch.manual_seed(42)
        np.random.seed(42)

        # Set device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {device}")

        # Load and prepare data
        metadata_df = pd.read_csv(r"/content/HAM10000_metadata.csv")
        logger.info(f"Loaded dataset with {len(metadata_df)} images")

        # Split data with stratification
        train_df, temp_df = train_test_split(
            metadata_df,
            test_size=0.3,
            stratify=metadata_df['dx'],
            random_state=42
        )

        val_df, test_df = train_test_split(
            temp_df,
            test_size=0.5,
            stratify=temp_df['dx'],
            random_state=42
        )

        logger.info(f"Train set: {len(train_df)} images")
        logger.info(f"Validation set: {len(val_df)} images")
        logger.info(f"Test set: {len(test_df)} images")

        # Create datasets
        train_dataset = SkinLesionDataset(
            image_dir=r"/content/HAM10000_images_part_main",
            mask_dir=r"/content/ham10000_segmentations/HAM10000_segmentations_lesion_tschandl",
            metadata_df=train_df,
            phase='train'
        )

        val_dataset = SkinLesionDataset(
            image_dir=r"/content/HAM10000_images_part_main",
            mask_dir=r"/content/segmented_images",
            metadata_df=val_df,
            phase='val'
        )

        test_dataset = SkinLesionDataset(
            image_dir=r"/content/HAM10000_images_part_main",
            mask_dir=r"/content/segmented_images",
            metadata_df=test_df,
            phase='val'
        )

        # Adjust number of workers based on system capability
        num_workers = 2  # Reduced from 4 to avoid the warning

        # Create data loaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )

        test_loader = DataLoader(
            test_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )

        # Initialize the VGG50 model (not SkinLesionDataset)
        model = VGG50(num_classes=7).to(device)  # Changed from SkinLesionDataset to VGG50

        model, history = train_model(
            train_loader=train_loader,
            val_loader=val_loader,
            model=model,
            num_epochs=10,
            device=device
        )

        # Evaluate model
        print("\nStarting final model evaluation...")
        test_metrics = evaluate_model(
            model=model,
            test_loader=test_loader,
            device=device
        )

        # Log final results
        print("\nFinal Results Summary:")
        print("=" * 80)
        print(f"Training Accuracy: {history['train_acc'][-1]:.2f}%")
        print(f"Validation Accuracy: {history['val_acc'][-1]:.2f}%")
        print(f"Test Accuracy: {test_metrics['accuracy']:.2f}%")
        print("=" * 80)

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()